In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

plt.style.use('default')

# Visualization

## Single-GPU Dump

In [ ]:
steps = [200, 400, 600]
vmin, vmax = 0, 0.05
Nx, Ny, Nz = 288, 272, 280
outputFolder = Path(f"left_inlet_right_outlet_cube_obs_{Nx}_{Ny}_{Nz}_output")

fig, axes = plt.subplots(1, len(steps), 
                         figsize=(8, 2.5),
                         sharey=True)

for i, step in enumerate(steps):
    vx_frame = np.fromfile(outputFolder / f"vx_{step}.dat", dtype=np.float32).reshape(Nz, Ny, Nx)
    vy_frame = np.fromfile(outputFolder / f"vy_{step}.dat", dtype=np.float32).reshape(Nz, Ny, Nx)
    vz_frame = np.fromfile(outputFolder / f"vz_{step}.dat", dtype=np.float32).reshape(Nz, Ny, Nx)
    v = np.sqrt(vx_frame**2 + vy_frame**2 + vz_frame**2)

    im = axes[i].imshow(v[:, Ny // 2, :], cmap="jet", origin="lower", vmin=vmin, vmax=vmax)
    axes[i].set_xlabel("X-axis")
    if i == 0:
        axes[i].set_ylabel("Z-axis")

    axes[i].set_title(f"Step {step}", fontsize=10)
    axes[i].set_xticks([])
    axes[i].set_yticks([])

fig.suptitle("Velocity Magnitude Slice at mid Y-plane", fontweight="bold", fontsize=12)
fig.subplots_adjust(wspace=0.1)

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), location="right", shrink=0.8, pad=0.04)
cbar.set_label(r"$\|\mathbf{v}\|$", rotation=0, labelpad=15)
cbar.ax.tick_params(labelsize=8)


fig.savefig("velocity_magnitude_slice.png", dpi=300, bbox_inches="tight")


## Multi-GPU Dump

Reconstruct the scalar field from a 2×2×1 device grid and visualize a Y-midplane slice.

In [ ]:
steps = [200, 400, 600, 800, 1000]
outputFolder = Path("multi_dev_output")
nx, ny, nz = (2, 2, 1)
Nx, Ny, Nz = (512, 512, 256)
Sx, Sy, Sz = Nx // nx, Ny // ny, Nz // nz

y_slice = Ny // 2

v_glob = np.empty((Nz, Ny, Nx), dtype=np.float32)

fig, axes = plt.subplots(
    len(steps),
    1,
    figsize=(4, 7),
    sharex=True,
)

for i, step in enumerate(steps):
    for dz in range(nz):
        for dy in range(ny):
            for dx in range(nx):
                vx = np.fromfile(outputFolder / f"vx_{step}_dev_{dx}_{dy}_{dz}.dat", dtype=np.float32).reshape((Sz, Sy, Sx))
                vy = np.fromfile(outputFolder / f"vy_{step}_dev_{dx}_{dy}_{dz}.dat", dtype=np.float32).reshape((Sz, Sy, Sx))
                vz = np.fromfile(outputFolder / f"vz_{step}_dev_{dx}_{dy}_{dz}.dat", dtype=np.float32).reshape((Sz, Sy, Sx))
                v_glob[
                    dz * Sz : (dz + 1) * Sz,
                    dy * Sy : (dy + 1) * Sy,
                    dx * Sx : (dx + 1) * Sx,
                ] = np.sqrt(vx**2 + vy**2 + vz**2)

    im = axes[i].imshow(
        v_glob[:, y_slice, :],
        cmap="jet",
        origin="lower",
        vmin=0,
        vmax=0.04,
        aspect="auto",
    )
    
    txt = axes[i].text(
        0.8, 0.85, f"Step {step}",
        transform=axes[i].transAxes,
        fontsize=8,
        weight="bold",
        color="w",
        va="top",
        ha="left",
    )

    # Add black outline to text
    txt.set_path_effects([
        pe.Stroke(linewidth=1.5, foreground="black"),
        pe.Normal(),
    ])


    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].set_ylabel("Z-axis")
    if i == len(steps) - 1:
        axes[i].set_xlabel("X-axis")

fig.suptitle(r"Velocity Magnitude at mid Y-plane", fontweight="bold", fontsize=12, y=0.98)

fig.subplots_adjust(
    left=0.04,
    right=0.96,
    top=0.93,
    bottom=0.16,
    hspace=0.05,
)

cbar = fig.colorbar(
    im,
    ax=axes.ravel().tolist(),
    orientation="horizontal",
    fraction=0.08,
    pad=0.09,
    ticks=np.linspace(0, 0.04, 5),
)
cbar.ax.xaxis.set_label_position('top')
cbar.ax.tick_params(labelsize=8)

cbar.set_label(r"$\|\mathbf{v}\|$")

fig.savefig("multi_dev_velocity_magnitude_slice.png", 
            dpi=300, bbox_inches="tight")
